In [1]:
import pandas as pd
import random

import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from spacy.lang.fr.examples import sentences 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\theoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_excel("shortEditionPosts.xlsx")

In [3]:
df

,title,genres,post,date,time,reads,likes,compet
0,Petits monstres,['Horreur'],Les poissons avaient cessé de mordre d'un seul...,2021-12-31 09:30:26,2,277\n,16,En compétition
1,L'Atelier de Maman,"['Instant de vie', 'Arts']","Quand on y entre, on aurait presque envie, d'a...",2021-12-27 09:30:05,2,532\n,22,En compétition
2,Une nouvelle chaussée,"['Instant de vie', 'Drame']","Du plat de la main, Achille caressa les cambru...",2021-12-26 09:30:04,3,283\n,26,En compétition
3,Ma vie où l'on voit la mer,"['Instant de vie', 'Société']",La plupart des gens ont une vie où l'on ne voi...,2021-12-24 09:30:25,3,667\n,57,En compétition
4,Juste un moment chez Sonia,"['Instant de vie', 'Humour']",Sonia adore ces soirées où elle invite des gen...,2021-12-22 09:30:05,3,1 027\n,57,En compétition
...,...,...,...,...,...,...,...,...
5765,Trois ballons,"['Nature', 'Suspense']","Trois ballons. Un rouge, un vert, un orange.\n...",2011-03-23 15:47:14,1,780\n,9,Qualifié
5766,La gardienne,"['Nature', 'Merveilleux']",Regardez un immense champ vert... Un champ ple...,2011-03-23 15:46:33,1,1 598\n,7,Qualifié
5767,Départ pour un calvaire,"['Famille', 'Instant de vie']","Ce matin, je me lève avec un poids insoutenabl...",2011-03-23 15:43:10,1,992\n,23,Qualifié
5768,Morceau classique,"['Instant de vie', 'Arts']",Madame se mit à jouer du piano. Ses doigts cou...,2011-03-23 10:00:00,1,4 141\n,123,Lauréat


In [4]:
stopwords = stopwords.words("French")

In [5]:
def delete_stopwords(df):
    text_out = []
    for texte_ in df['post']:
        texte = ''.join(str(e) for e in texte_)
        mots = texte.split()
        final_mots = []
        for mot in mots:
            if mot not in stopwords:
                final_mots.append(mot)
            
        text_out.append(' '.join(str(mot) for mot in final_mots))
    return(text_out)
data = delete_stopwords(df)


In [6]:
data

["Les poissons cessé mordre d'un seul coup... À surface l'étang, régnait calme anormal. Aucune vaguelette, aucune bulle venue fond, comme si vie cessé brutalement là-dessous... Tom frère Jules savaient très bien cela signifiait... — Passe-moi boite, s'il plaît ! dit Tom. — Tiens ! — Non celle-ci, donne-moi petits rouges. Ils plus nerveux. Tom ouvrit couvercle petite boite découvrit satisfaction amas vers grouillants s'agitaient tous sens. — Regarde comme beaux. Avec ça, vais faire malheur, annonça-t-il certitude frère. Il fouilla doigts sciure, sentit alors petites bêtes chatouiller creux main. Il aimait vraiment cette sensation. Il prit tout temps choisir l'asticot idéal. Celui auquel aucun poisson résisterait. Tom Jules s'étaient spécialisés l'élevage asticots. Les petits, gros, joufflus, gras, nerveux, colorés... Rien manquait panoplie. Ils jusqu'à jour tout étudié, tout essayé... Enfin, presque... Ils commencé faire pourrir poisson, celui qu'ils pêché l'étang. Ensuite, s'étaient mi

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords)
matrix = vectorizer.fit_transform(df['post'])

In [8]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=6, metric='cosine').fit(matrix)

def content_based_recommender(title):
    text_index = df[df["title"]==title].index.values[0]
    distances, indices = neighbors.kneighbors(matrix[text_index])
    for index in indices[0][1:]:
        print(df.loc[index]["title"])

In [9]:
def print_text(title):
    text_index = df[df["title"]==title].index.values[0]
    print(df.loc[text_index]["post"])

In [10]:
content_based_recommender("L'Atelier de Maman")

Vie d'odeurs
Lumière et vieux papier
En plein minuit
La chambre mystérieuse
Ordinaire


In [11]:
matrix[0]

<1x82439 sparse matrix of type '<class 'numpy.float64'>'
	with 312 stored elements in Compressed Sparse Row format>

## Remarques

TF-idf semble donner beaucoup d'importance aux noms propres (propose des textes avec le même nom pour le personnage principal). Pas idéal, faudrait voir si on peut contourner ça.

# Système de recommandations

In [12]:
%reset -f

In [13]:
import pandas as pd
import random

import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from spacy.lang.fr.examples import sentences 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\theoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from IPython import display
from ipywidgets import AppLayout, Button, GridspecLayout, Image, Layout, Textarea, HTML
from sklearn.model_selection import train_test_split

In [15]:
df = pd.read_excel("shortEditionPosts.xlsx")

In [16]:
stopwords = stopwords.words("French")

In [17]:
train_ops, test_ops = train_test_split(df, test_size=0.2)

In [18]:
train_ops = train_ops.reset_index(drop=True)
test_ops = test_ops.reset_index(drop=True)

In [19]:
# Add empty column to record actions
train_ops['interested'] = None
test_ops['interested'] = None



In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
vectorizer = TfidfVectorizer(stop_words=stopwords)
train_matrix = vectorizer.fit_transform(train_ops['post'])

In [21]:
def create_expanded_button(description, button_style):
    return Button(
        description=description,
        button_style=button_style,
        layout=Layout(height='auto', width='auto')
    )


def random_row():
    return random.randint(0, len(train_ops) - 1)


def get_post(index, dataset):
    title = "<h1>"+dataset.loc[index]['title']+"</h1>"
    post = dataset.loc[index]['post']
    return HTML(
    value=title+post
    )


def plot_grid(df):
    n_rows = (len(df)//3) + (len(df) % 3 != 0)
    grid = GridspecLayout(n_rows , 3)

    cpt = 0
    for i in range(n_rows):
        for j in range(3):
            if cpt < len(df):
                grid[i, j] = get_post(df.index[cpt], df)
                cpt += 1 
    return grid

In [22]:
# Setting the dislike button: Not Interested
dislike_button = create_expanded_button('Not Interested', 'danger')

# Setting the post in the center
current_row_number = random_row()
post_widget = get_post(current_row_number, train_ops)

# Setting the like button: Interested
like_button = create_expanded_button('Interested', 'success')

EPSILON = 0.1

def on_button_clicked(b):
    """
    Update the values related to the users choice
    Choose a new operation to display
    Update the banner displayed
    """
    update_values(b)
    row_number = choose_row_number()
    update_post(b, row_number)
    

def update_values(b):
    """
    Update the dataframe column "interested" using user's action
    """
    global current_row_number

    interested = b.description == "Interested"
    train_ops.loc[current_row_number, 'interested'] = interested
    

def choose_row_number():
    """
    We take an epsilon greedy approach
    """
    if random.random() < EPSILON or not train_ops[~train_ops.interested.isna()].shape[0] > 2:
        # make a random prediction
        best_row = random_row()
        if train_ops.loc[best_row].interested is None:
            return best_row    
    
        while(train_ops.loc[row_number].interested is not None):
            best_row = random_row()
        
    else :
        # we make a prediction based on what the user has already seen
        #indexes of already seen posts
        train_current = train_ops.copy()
        seen_indexes = train_ops[~train_ops.interested.isna()].index.values
        
        X = train_matrix[seen_indexes]
        y = list(train_ops.loc[seen_indexes].interested)

        clf = LogisticRegression().fit(X, y)

        prediction = clf.predict_proba(train_matrix)[:,1]

        train_current['prediction'] = prediction
        
        best_row = train_current[train_current.interested.isna()].sort_values(['prediction'], ascending=False).index.values[0]

    
    return best_row


def update_post(b, row_number):
    """
    Update the value of the image widget with the new banner's string
    """
    global current_row_number
    current_row_number = row_number
    post = get_post(row_number, train_ops)
    post_widget.value = post.value
    

# Set the on_click function to the button
dislike_button.on_click(on_button_clicked)
like_button.on_click(on_button_clicked)

# https://ipywidgets.readthedocs.io/en/stable/examples/Layout%20Templates.html#AppLayout
"""
AppLayout(
    left_sidebar=left_button,
    center=post,
    right_sidebar=right_button
)
"""
main_grid = GridspecLayout(1, 5)
like_grid = GridspecLayout(2,1)
like_grid[0,:] = like_button
like_grid[1, :] = dislike_button
main_grid[:, :4] = post_widget
main_grid[:, 4] = like_grid

In [23]:
main_grid

GridspecLayout(children=(HTML(value='<h1>La partie de pétanque</h1>Les gens ne m’aiment pas. Cela remonte à mo…